In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

api_key = os.environ["OPENAI_API_KEY"] 

In [2]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, HuggingFaceInstructEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.indexes import VectorstoreIndexCreator
from langchain_experimental.agents.agent_toolkits.csv.base import create_csv_agent
from langchain.agents.agent_types import AgentType
from langchain.memory import ConversationBufferMemory
import tiktoken


In [3]:
from langchain.text_splitter import CharacterTextSplitter

In [4]:
txt_file_path = "/home/lillian/Documents/TenAcademy/week11/Contract-AI-Chatbot/backend/data/contract.txt"

In [5]:
loader = TextLoader(file_path= txt_file_path, encoding="utf-8")
data = loader.load()

In [6]:
print(type(loader))

<class 'langchain_community.document_loaders.text.TextLoader'>


In [6]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
data = text_splitter.split_documents(data)

Created a chunk of size 1682, which is longer than the specified 1000
Created a chunk of size 2768, which is longer than the specified 1000
Created a chunk of size 3044, which is longer than the specified 1000
Created a chunk of size 3193, which is longer than the specified 1000
Created a chunk of size 3845, which is longer than the specified 1000
Created a chunk of size 2978, which is longer than the specified 1000
Created a chunk of size 3343, which is longer than the specified 1000
Created a chunk of size 3880, which is longer than the specified 1000
Created a chunk of size 1864, which is longer than the specified 1000
Created a chunk of size 3444, which is longer than the specified 1000
Created a chunk of size 2859, which is longer than the specified 1000
Created a chunk of size 3258, which is longer than the specified 1000
Created a chunk of size 1665, which is longer than the specified 1000
Created a chunk of size 2288, which is longer than the specified 1000
Created a chunk of s

In [8]:
print(type(data))

<class 'list'>


In [7]:
embeddings = OpenAIEmbeddings()

/home/lillian/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [8]:
vectorstore = FAISS.from_documents(data, embedding=embeddings)

In [9]:
llm = ChatOpenAI(temperature=0.1, model_name="gpt-4",max_tokens=500)

/home/lillian/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [10]:
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
# from langchain.llms import OpenAI

chain = load_qa_with_sources_chain(llm=llm, chain_type="map_rerank")
query = "What is the escrow amount?"
chain({"input_documents": data, "question": query}, return_only_outputs=True)

/home/lillian/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/home/lillian/.local/lib/python3.10/site-packages/langchain/chains/llm.py:344: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


KeyboardInterrupt: 

In [14]:
from langchain.chains import RetrievalQAWithSourcesChain

new_chain = RetrievalQAWithSourcesChain(
    return_source_documents=True,
    verbose=True,
    combine_documents_chain= chain,
    retriever=vectorstore.as_retriever(),
)

In [12]:
query = "What is the escrow amount?"
new_chain({"question": query})



> Entering new RetrievalQAWithSourcesChain chain...


/home/lillian/.local/lib/python3.10/site-packages/langchain/chains/llm.py:344: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(



> Finished chain.


{'question': 'What is the escrow amount?',
 'answer': 'This document does not provide information on the escrow amount.',
 'sources': '',
 'source_documents': [Document(page_content='Escrow.', metadata={'source': '/home/lillian/Documents/TenAcademy/week11/Contract-AI-Chatbot/backend/data/contract.txt'}),
  Document(page_content='(a)\nAt Closing, Buyer will deposit the Escrow Amount in escrow on behalf of the\nSellers in accordance with the Escrow Agreement. The Escrow Amount shall be held and,\nsubject to Section 2.07, released to the Company Securityholders in accordance with the\nprovisions of the Escrow Agreement with the Company Securityholders being entitled to share in\nsuch released amounts in accordance with their Pro Rata Percentages. From and after the\nClosing, Buyer and the Sellers’ Representative will direct the Escrow Agent to disburse\npayments from the Escrow Account in accordance with the purchase price adjustment provisions\nof this Agreement and the terms of the Escr

In [26]:
from langchain.chains import (
    StuffDocumentsChain,
    LLMChain,
    ReduceDocumentsChain,
    MapReduceDocumentsChain,
)
from langchain_core.prompts import PromptTemplate


document_prompt = PromptTemplate(
    input_variables=["page_content"],
     template="{page_content}"
)
document_variable_name = "context"


prompt = PromptTemplate.from_template(
    "Summarize this content: {context}"
)
llm_chain = LLMChain(llm=llm, prompt=prompt,verbose= True)

reduce_prompt = PromptTemplate.from_template(
    "Combine these summaries: {context}"
)
reduce_llm_chain = LLMChain(llm=llm, prompt=reduce_prompt,verbose= True)
combine_documents_chain = StuffDocumentsChain(
    llm_chain=reduce_llm_chain,
    document_prompt=document_prompt,
    document_variable_name=document_variable_name
)
reduce_documents_chain = ReduceDocumentsChain(
    combine_documents_chain=combine_documents_chain,verbose=True
)
chain = MapReduceDocumentsChain(
    llm_chain=llm_chain,
    reduce_documents_chain=reduce_documents_chain,verbose=True
)

prompt = PromptTemplate.from_template(
    "Collapse this content: {context}"
)
llm_chain = LLMChain(llm=llm, prompt=prompt)
collapse_documents_chain = StuffDocumentsChain(
    llm_chain=llm_chain,
    document_prompt=document_prompt,
    document_variable_name=document_variable_name,verbose=True
)
reduce_documents_chain = ReduceDocumentsChain(
    combine_documents_chain=combine_documents_chain,
    collapse_documents_chain=collapse_documents_chain,verbose= True
)
chain = MapReduceDocumentsChain(
    llm_chain=llm_chain,
    reduce_documents_chain=reduce_documents_chain,
    verbose=True
)

In [27]:
query = "What is the escrow amount?"
chain({"question":query,"input_documents": data})



> Entering new MapReduceDocumentsChain chain...


KeyboardInterrupt: 

In [21]:
query = " Who are the people in the agreement"
chain({"input_documents": data, "question": query}, return_only_outputs=True)

/home/lillian/.local/lib/python3.10/site-packages/langchain/chains/llm.py:344: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


{'output_text': "The people in the agreement are the buyer, the target company, the sellers listed on Schedule I, and the sellers' representative."}

In [52]:
from langchain.chains import StuffDocumentsChain, LLMChain
from langchain_core.prompts import PromptTemplate
from langchain_community.llms import OpenAI
           # This controls how each document will be formatted. Specifically,
           # it will be passed to `format_document` - see that function for more
# details.
document_prompt = PromptTemplate(
    input_variables=["page_content"],
    template="{page_content}"
)
document_variable_name = "context"
llm = OpenAI()
# The prompt here should take as an input variable the
# `document_variable_name`
prompt = PromptTemplate.from_template(
    "Summarize this content: {context}"
)
llm_chain = LLMChain(llm=llm, prompt=prompt)
chain = StuffDocumentsChain(
    llm_chain=llm_chain,
    document_prompt=document_prompt,
    document_variable_name=document_variable_name
)
   

In [54]:
query = "How much is the escrow amount?"
result = chain({"question": query, "input_documents": data})
answer = result["answer"]
answer

BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 4097 tokens, however you requested 48419 tokens (48163 in your prompt; 256 for the completion). Please reduce your prompt; or completion length.", 'type': 'invalid_request_error', 'param': None, 'code': None}}

In [49]:
from langchain.chains import (
    StuffDocumentsChain, LLMChain, ConversationalRetrievalChain
)
from langchain_core.prompts import PromptTemplate
from langchain_community.llms import OpenAI
from langchain.memory import ConversationBufferWindowMemory

document_prompt = PromptTemplate(
    input_variables=["page_content"],
    template="{page_content}"
)
llm_chain = LLMChain(llm=llm, prompt=document_prompt)
combine_docs_chain = StuffDocumentsChain(llm_chain=llm_chain, document_prompt=document_prompt)
retriever = vectorstore.as_retriever() 
# This controls how the standalone question is generated.
# Should take `chat_history` and `question` as input variables.
template = (
    "Combine the chat history and follow up question into "
    "a standalone question. Chat History: {chat_history}"
    "input: {context}"
    "Follow up question: {question}"
)
prompt = PromptTemplate.from_template(template)
memory = ConversationBufferWindowMemory(memory_key="chat_history", k=1) 
question_generator_chain = LLMChain(llm=llm, prompt=prompt)
chain = ConversationalRetrievalChain(
    combine_docs_chain=combine_docs_chain,
    retriever=retriever,
    question_generator=question_generator_chain,
    memory= memory
)

In [50]:
query = "What is the document about?"

result = chain({"question": query})

answer = result["answer"]

answer

'\n\n\n\nSection 9.11\n\nWaiver of Jury Trial\n\n49\n\nSection 9.12\n\nSpecific Performance.\n\n50\n\nSection 9.13\n\nEffect of Headings\n\n50\n\nSection 9.14\n\nFees and Expenses\n\n50\n\nSection 9.15\n\nNon-Recourse\n\n50\n\nSection 9.16\n\nInterpretation\n\n50\n\nSection 9.17\n\nConstruction\n\n50\n\nSection 9.18\n\nNo Presumption Against Drafting Party\n\n51\n\nSection 9.19\n\nRemedies Cumulative\n\n51\n\nSection 9.20\n\nFurther Assurances\n\n51\n\nSection 9.21\n\nRemedies; Attorney’s Fees\n\n51\n\nSection 9.22\n\nDisclosure Schedules\n\n51\n\nSection 9.23\n\nAdditional Representations and Warranties\n\n51\n\nSection 9.24\n\nConflicts\n\n51\n\n-3-\n\n112923184_5\n\n\x0cARTICLE I\nDEFINITIONS\nSection 1.01\n\nCertain Defined Terms\n\n. As used in this Agreement (including the Recitals hereto), the\nfollowing terms shall have the meanings specified below:\n\n“\n\nAcquired Entity\n\n” means the Entities that are being acquired by Buyer pursuant to Section'

In [51]:
from IPython.display import display, Markdown

formatted_answer = f"**Answer:** \n {answer}"
display(Markdown(formatted_answer))

**Answer:** 
 



Section 9.11

Waiver of Jury Trial

49

Section 9.12

Specific Performance.

50

Section 9.13

Effect of Headings

50

Section 9.14

Fees and Expenses

50

Section 9.15

Non-Recourse

50

Section 9.16

Interpretation

50

Section 9.17

Construction

50

Section 9.18

No Presumption Against Drafting Party

51

Section 9.19

Remedies Cumulative

51

Section 9.20

Further Assurances

51

Section 9.21

Remedies; Attorney’s Fees

51

Section 9.22

Disclosure Schedules

51

Section 9.23

Additional Representations and Warranties

51

Section 9.24

Conflicts

51

-3-

112923184_5

ARTICLE I
DEFINITIONS
Section 1.01

Certain Defined Terms

. As used in this Agreement (including the Recitals hereto), the
following terms shall have the meanings specified below:

“

Acquired Entity

” means the Entities that are being acquired by Buyer pursuant to Section

### NOT ACCURATE AND VERY SLOW SPEED 